In [61]:
import torch

from utils import load_epsilon_net, load_image
from utils import load_epsilon_net
from sampling.dps import dps
from sampling.dps_dpms import dps_dpms
from time import time

device = "cuda:0"
#n_steps = 100
torch.set_default_device(device)


# load the image
img_path = "./material/celebahq_img/00010.jpg"
x_origin = load_image(img_path, device)


# load the degradation operator
#path_operator = f"./material/degradation_operators/sr16.pt"
path_operator = f"./material/degradation_operators/outpainting_half.pt"
degradation_operator = torch.load(path_operator, map_location=device)

# apply degradation operator
y = degradation_operator.H(x_origin[None])
y = y.squeeze(0)

sigma = [0.01, 0.05, 0.1]
n_steps = [100, 300, 500, 1000]


method = "dps_dpms"

output_dir= "./output"



for n in n_steps:
    for s in sigma:
        K = [int(n/10), int(n/5), int(n/2)]
        for k in K:

            # add noise
            y = y + s * torch.randn_like(y)

            # define inverse problem
            inverse_problem = (y, degradation_operator, s)

            # load model
            eps_net = load_epsilon_net("celebahq", n, device)

            # solve problem
            initial_noise = torch.randn((1, 3, 256, 256), device=device)
            # start_time = time()
            # #reconstruction = dps(initial_noise, inverse_problem, eps_net)
            # reconstruction = dps_dpms(initial_noise, inverse_problem, eps_net, k = k)
            # #reconstruction = dpms(initial_noise, inverse_problem, eps_net)
            # end_time = time()

            start = time()
            if method == "dpms":
                reconstruction = dpms(initial_noise, inverse_problem, eps_net)
                output_path = output_dir + f"/{method}_n_step={n}_sigma={s}.png"
            if method == "dps":
                reconstruction = dps(initial_noise, inverse_problem, eps_net)
                output_path = output_dir + f"/{method}_n_step={n}_sigma={s}.png"
            if method == "dps_dpms":
                reconstruction = dps_dpms(initial_noise, inverse_problem, eps_net,k=k)
                output_path = output_dir + f"/{method}_n_step={n}_sigma={s}_k={k}.png"
            end = time()
            print(f"Time: {end-start}")
            reconstruction.clamp(-1, 1)
            #display the results
            # reshape y
            y_reshaped =  -torch.ones(3 * 256 * 256, device=device)
            y_reshaped[: y.shape[0]] = y
            y_reshaped = degradation_operator.V(y_reshaped[None])
            y_reshaped = y_reshaped.reshape(3, 256, 256)

            #metric calculation
            lpips =  LPIPS()
            lpips_score = lpips.score(reconstruction.clamp(-1, 1), x_origin)

            # init figure
            fig, axes = plt.subplots(1, 3)

            images = (x_origin, y_reshaped, reconstruction[0])
            titles = ("original", "degraded", "reconstruction")

            # display figures
            
            for ax, img, title in zip(axes, images,titles):
                display_image(img, ax)
                ax.set_title(title)
            if method == "dps_dpms":
                fig.suptitle(f"{method}, n_steps={n}, s={s}, k={k}, lpips={round(lpips_score.item(),2)}, time={round(end-start,2)}")
            else:
                fig.suptitle(f"{method}, n_steps={n}, s={s}, lpips={round(lpips_score.item(),2)}, time={round(end-start,2)}")
            fig.tight_layout()
            fig.savefig(output_path)
            plt.close(fig)
            #print("Time elapsed: ", end_time - start_time)

diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  3.73it/s]
c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\torch\utils\_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


Time: 9.415191173553467
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\lpips\weights\v0.1\alex.pth


c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739

Time: 9.02949833869934
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\lpips\weights\v0.1\alex.pth


diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  2.77it/s]


...

Time: 93.19001078605652
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\lpips\weights\v0.1\alex.pth


diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  3.92it/s]


Time: 89.6328330039978
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\lpips\weights\v0.1\alex.pth


diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  3.82it/s]


Time: 79.74686074256897
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Dolly\Documents\My_Files\code\hackathon_starter_kit\venv-hackathon\Lib\site-packages\lpips\weights\v0.1\alex.pth
